In [1]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

persist_directory = "/Users/mariaborca/Documents/AI_2023-2024/Semestrul 4/Machine Learning/Know-Your-Rights/chroma_db_articles"
vector_store = Chroma(
    collection_name="know_your_rights",
    embedding_function=OpenAIEmbeddings(api_key=OPENAI_API_KEY),
    persist_directory=persist_directory
)

def retrive_relevant_documents(query: str, vector_store: Chroma, k: int = 5):
    results = vector_store.similarity_search_with_score(query, k=k)
    results.sort(key=lambda x: x[1], reverse=False)
    return results

def get_openai_llm(model: str = "gpt-4o-mini"):
    return OpenAI(
        model=model,
        temperature=0,
        max_tokens=1000,
        api_key=OPENAI_API_KEY)

def get_local_model(model: str = "phi-4"):
    base_url = "http://localhost:1234/v1"
    api_key = "lm-studio"

    return ChatOpenAI(
        base_url=base_url,
        api_key=api_key,
        temperature=0.9,
        model=model
    )

def get_response(query, llm, template=None, context=None):
    if template is None:
        template = """
        Folosește **doar informațiile din contextul de mai jos** pentru a răspunde la întrebare. 
        Dacă răspunsul nu este prezent în context, răspunde cu „Nu știu”.

        Context:
        {context}

        Întrebare:
        {question}

        Răspuns:

        """
        
    prompt = template.format(context=context, question=query)
    response = llm.invoke(prompt)
    return response.content if type(response)!=str else llm.invoke(prompt)

def load_test_cases(file_path):
    with open(file_path, 'r') as file:
        test_cases = json.load(file)
    return test_cases

In [15]:
from deepeval import evaluate
from deepeval.metrics import GEval, AnswerRelevancyMetric, ContextualRecallMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.evaluate.configs import DisplayConfig

import io
import contextlib

def extract_metric_info(evaluation_result):
    results = []
    for test_result in evaluation_result.test_results:
        for metric_data in test_result.metrics_data:
            results.append({
                "metric": metric_data.name,
                "success": metric_data.success,
                "score": metric_data.score,
                "reason": metric_data.reason
            })
    return results

silent_display_config = DisplayConfig(
    print_results=False,
    show_indicator=False
)

def test_answer_relevancy(user_query, actual_answer, expected_output):
    metric = AnswerRelevancyMetric(
        threshold=0.7,
        include_reason=True,
        verbose_mode=False,
    )

    test_case = LLMTestCase(
        input=user_query,
        actual_output=actual_answer,
        expected_output=expected_output
        )
    
    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def test_contextual_recall(user_query, expected_answer, actual_answer, retrival_context):
    metric = ContextualRecallMetric(
        threshold=0.7,
        include_reason=True,
        verbose_mode=False
    )
    test_case = LLMTestCase(
        input=user_query,
        expected_output=expected_answer,
        actual_output=actual_answer,
        retrieval_context=retrival_context
    )

    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def test_faithfulness(user_query, actual_answer, retrival_context, expected_output):
    metric = FaithfulnessMetric(
        threshold=0.7,
        verbose_mode=False,
        model="gpt-4o-mini",
    )
    test_case = LLMTestCase(
        input=user_query,
        actual_output=actual_answer,
        retrieval_context=retrival_context,
        expected_output=expected_output
    )
    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def test_understanding(user_query, expected_answer, actual_answer):
    metric = GEval(
        name="Understanding",
        evaluation_steps=(
            ["Răspunsul este formulat într-un mod clar și accesibil pentru o persoană fără cunoștințe juridice? ",
            "Poate utilizatorul să înțeleagă ce are voie să facă sau ce trebuie să facă, pe baza acestui răspuns?"]
        ),
        threshold=0.5,
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
        verbose_mode=False,
        model="gpt-4o-mini",
    )
    test_case = LLMTestCase(
        input=user_query,
        expected_output=expected_answer,
        actual_output=actual_answer
    )
    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def test_toxicity(user_query, actual_answer):
    metric = GEval(
        name="Toxicity",
        evaluation_steps=(
            ["Răspunsul conține un limbaj jignitor, rasist, sexist sau altfel inadecvat? ",
            "Răspunsul este formulat într-un mod care ar putea fi considerat ofensator sau inadecvat?"]
        ),
        threshold=0.7,
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
        verbose_mode=False,
        model="gpt-4o-mini",
    )
    test_case = LLMTestCase(
        input=user_query,
        actual_output=actual_answer
    )
    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def run_evaluation(test_cases, vector_store, llm, prompt):
    results = []
    for test_case in test_cases:
        user_query = test_case['question']
        expected_answer = test_case['expected_answer']

        retrival_context = retrive_relevant_documents(user_query, vector_store, k=3)
        retrival_context = [doc.page_content for doc, _ in retrival_context]
        context = "\n\n".join(retrival_context)
        
        # Run RAG to get the actual answer
        actual_answer = get_response(user_query, llm, prompt, context)

        # Evaluate the answers
        relevancy_result = test_answer_relevancy(user_query, actual_answer, expected_answer)
        recall_result = test_contextual_recall(user_query, expected_answer, actual_answer, retrival_context)
        faithfulness_result = test_faithfulness(user_query, actual_answer, retrival_context, expected_answer)
        understanding_result = test_understanding(user_query, expected_answer, actual_answer)
        toxicity_result = test_toxicity(user_query, actual_answer)

        relevancy_result = extract_metric_info(relevancy_result)
        recall_result = extract_metric_info(recall_result)
        faithfulness_result = extract_metric_info(faithfulness_result)
        understanding_result = extract_metric_info(understanding_result)
        toxicity_result = extract_metric_info(toxicity_result)

        print(f"""
            Query: {user_query}
            Expected: {expected_answer}
            Actual: {actual_answer}
            Relevancy Score: {relevancy_result}
            Recall Score: {recall_result}
            Faithfulness Score: {faithfulness_result}
            Understanding Score: {understanding_result}
            Toxicity Score: {toxicity_result}
            """)
       

        results.append({
            'user_query': user_query,
            'expected_answer': expected_answer,
            'actual_answer': actual_answer,
            'relevancy': relevancy_result,
            'recall': recall_result,
            'faithfulness': faithfulness_result,
            'understanding': understanding_result,
            'toxicity': toxicity_result
        })
    
    return results

def start_testing(models_list, test_cases, prompt):
    results_for_models = {model : [] for model in models_list}
    for test in test_cases:
        print(f"Running tests for query: {test['question']}")
        for model in models_list:
            print(f"Using model: {model}")
            llm = get_local_model(model=model)
            results = run_evaluation([test], vector_store, llm, prompt)
            results_for_models[model].append(results)
        print(f"Finished tests for model: {model}\n")
        print("=" * 50)
        print("\n")
    return results_for_models


from statistics import mean, stdev
from collections import defaultdict

def analyze_model_results(model_results):
    """
    Input: model_results - list (or list of lists) of dictionaries with keys:
        'user_query', 'expected_answer', 'actual_answer', 'relevancy', 'recall', 'faithfulness', 'understanding'

    Output:
        - average score per metric
        - pass rate per metric
        - a unified list of examples (one per test case, includes all metrics)
    """

    # Flatten if it's a list of lists
    if any(isinstance(item, list) for item in model_results):
        flat_results = [entry for sublist in model_results for entry in sublist]
    else:
        flat_results = model_results

    metric_data = defaultdict(lambda: {
        "scores": [],
        "successes": 0,
        "total": 0
    })

    unified_examples = []

    for result in flat_results:
        test_example = {
            "query": result["user_query"],
            "expected": result["expected_answer"],
            "actual": result["actual_answer"],
            "metrics": {}
        }

        for metric_name in ["relevancy", "recall", "faithfulness", "understanding"]:
            metric_entries = result.get(metric_name, [])
            for entry in metric_entries:
                name = entry["metric"]
                score = entry["score"]
                success = entry["success"]
                reason = entry.get("reason", "")

                # Track overall stats
                metric_data[name]["scores"].append(score)
                metric_data[name]["total"] += 1
                if success:
                    metric_data[name]["successes"] += 1

                # Store this metric result under the test case
                test_example["metrics"][name] = {
                    "score": score,
                    "success": success,
                    "reason": reason
                }

        unified_examples.append(test_example)

    summary = {}
    for metric, data in metric_data.items():
        scores = data["scores"]
        summary[metric] = {
            "avg_score": round(mean(scores), 3) if scores else None,
            "std_dev": round(stdev(scores), 3) if len(scores) > 1 else 0.0,
            "pass_rate": f"{round((data['successes'] / data['total']) * 100, 1)}%" if data["total"] > 0 else "N/A",
            "total_cases": data["total"]
        }

    # Add unified examples only once
    summary["examples"] = unified_examples

    return summary

tests_list = load_test_cases("test_examples.json")


In [4]:
models_list = ["rollama3-8b-instruct", "rollama2-7b-instruct"]

prompt_v2 = """
Ești un ghid juridic virtual, care ajută cetățenii să înțeleagă legea pe înțelesul tuturor. 

Când răspunzi, urmează aceste reguli:
 - Explică termenii juridici într-un mod clar, ca și cum ai vorbi cu cineva fără pregătire juridică.
 - Fii concis, dar oferă informații corecte și complete.
 - Nu cita articole de lege și nu inventa numere de articole.
 - Scrie răspunsul în propoziții scurte și clare. Poți folosi listă cu puncte, dacă este nevoie.
 - Nu inventa informații. Dacă informația nu se regăsește clar în contextul oferit, răspunde cu: **„Nu am putut genera un răspuns.”**
 - Nu oferi sfaturi legale personalizate; explică doar cadrul legal general.

Întrebarea utilizatorului este: {question}

Folosește următoarele informații extrase din documentele juridice:
{context}

Răspunsul tău trebuie să fie în limba română, clar, politicos și ușor de înțeles.
"""

model_results = start_testing(models_list, tests_list, prompt_v2)

for model_name, model_result_list in model_results.items():
    print(f"Evaluating model: {model_name}")
    results_summary = analyze_model_results(model_result_list)
    print(json.dumps(results_summary, indent=4, ensure_ascii=False))

Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: rollama3-8b-instruct


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?
            Expected: În cazul concedierii pentru motive care nu țin de persoana salariatului, angajatorul trebuie să respecte proceduri stricte și să asigure drepturi specifice salariatului. Concedierea este permisă doar pentru desființarea locului de muncă din motive economice, tehnologice, structurale sau similare. Angajatorul are obligația să acorde un preaviz de minimum 20 de zile lucrătoare și să respecte criterii obiective de selecție (performanțe profesionale, vechime, sarcini de întreținere).
            Actual: În ceea ce privește drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului, putem enumera următoarele aspecte legale:

1. Concedierea se poate efectua pentru orice motiv care nu are legătură cu salariatul. Aceasta include desființarea postului ocupat 

In [6]:
prompt_v3 = """ 
Ești un ghid juridic virtual. Scopul tău este să explici legea în limba română într-un mod clar, concis și ușor de înțeles pentru orice cetățean, fără a folosi termeni tehnici sau limbaj complicat.

Când răspunzi, respectă aceste reguli:
 - Răspunsul trebuie să conțină propoziții scurte, clare și fără termeni juridici complicați. Evită frazele lungi.
 - Explică termenii juridici pe înțelesul oricui, fără a presupune cunoștințe legale.
 - Nu cita articole de lege și nu inventa surse sau exemple.
 - Folosește doar informațiile din context. Nu completa cu informații suplimentare.
 - Dacă contextul nu oferă un răspuns clar, scrie simplu: **„Nu am putut genera un răspuns.”**
 - Fii concis. Nu scrie mai mult decât este necesar pentru a răspunde clar la întrebare.
 - Evită limbajul prea tehnic sau abstract. Fii prietenos, dar neutru.

Întrebarea utilizatorului este:  
{question}

Informațiile disponibile sunt:  
{context}

Scrie răspunsul în limba română. Acesta trebuie să fie clar, politicos și ușor de înțeles.
"""
model_results = start_testing(models_list, tests_list, prompt_v3)

for model_name, model_result_list in model_results.items():
    print(f"Evaluating model: {model_name}")
    results_summary = analyze_model_results(model_result_list)
    print(json.dumps(results_summary, indent=4, ensure_ascii=False))

Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: rollama3-8b-instruct


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?
            Expected: În cazul concedierii pentru motive care nu țin de persoana salariatului, angajatorul trebuie să respecte proceduri stricte și să asigure drepturi specifice salariatului. Concedierea este permisă doar pentru desființarea locului de muncă din motive economice, tehnologice, structurale sau similare. Angajatorul are obligația să acorde un preaviz de minimum 20 de zile lucrătoare și să respecte criterii obiective de selecție (performanțe profesionale, vechime, sarcini de întreținere).
            Actual:  
Concedierea individuală pentru motive care nu țin de persoana salariatului este reglementată de articolul 61 din Codul muncii românesc, care stipulează că angajatorii au dreptul de a-și desființa locurile de muncă. Cu toate acestea, concedierea trebuie să fie efectivă și să aibă o cauză reală și serioasă. În plus, a

In [7]:
prompt_v4 = """ 
Ești un ghid juridic virtual. Scopul tău este să explici legea în limba română într-un mod clar, concis și accesibil oricărui cetățean, fără a folosi termeni tehnici sau limbaj complicat.

Respectă cu strictețe următoarele reguli:
 - Scrie propoziții scurte, clare și ușor de înțeles. Evită frazele lungi și ambigue.
 - Nu folosi termeni juridici specializați. Dacă apar în context, explică-i simplu.
 - Nu inventa informații și nu completa cu exemple din cunoștințele tale. Fii fidel exclusiv contextului.
 - Nu cita articole de lege și nu menționa surse sau numere de articole.
 - Dacă informația necesară nu se găsește în context, scrie exact: **„Nu am putut genera un răspuns.”**
 - Evită generalizările. Limitează-te doar la ceea ce este prezent în context.
 - Răspunsul trebuie să fie scurt, complet și fără comentarii inutile.
 - Păstrează un ton politicos, neutru și prietenos. Nu oferi sfaturi legale personalizate.

Uite două exemple de întrebări și răspunsuri pentru a înțelege ce se așteaptă:

Exemplu bun:  
Întrebare: Ce protecție oferă statul român cetățenilor săi aflați în afara țării?  
Răspuns: Cetățenii români aflați în străinătate beneficiază de protecția statului român. Ei trebuie să-și respecte obligațiile, cu excepția celor care nu pot fi îndeplinite din cauza absenței din țară.

Exemplu greșit:  
Întrebare: Ce drepturi are un chiriaș?  
Răspuns: În general, chiriașii au dreptul la o locuință decentă, iar proprietarul nu are voie să-i deranjeze. Dacă ceva nu merge bine, e suficient ca chiriașul să notifice proprietarul pentru a pleca.  
(Motive: răspunsul este vag, incomplet și conține informații care nu sunt în contextul oferit.)
---

Întrebarea utilizatorului este:  
{question}

Informațiile disponibile sunt:  
{context}

Scrie răspunsul în limba română. Acesta trebuie să fie clar, politicos și ușor de înțeles.
"""

model_results = start_testing(models_list, tests_list, prompt_v3)

for model_name, model_result_list in model_results.items():
    print(f"Evaluating model: {model_name}")
    results_summary = analyze_model_results(model_result_list)
    print(json.dumps(results_summary, indent=4, ensure_ascii=False))

Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: rollama3-8b-instruct


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?
            Expected: În cazul concedierii pentru motive care nu țin de persoana salariatului, angajatorul trebuie să respecte proceduri stricte și să asigure drepturi specifice salariatului. Concedierea este permisă doar pentru desființarea locului de muncă din motive economice, tehnologice, structurale sau similare. Angajatorul are obligația să acorde un preaviz de minimum 20 de zile lucrătoare și să respecte criterii obiective de selecție (performanțe profesionale, vechime, sarcini de întreținere).
            Actual: 
În ceea ce privește drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului, acestea includ următoarele elemente:
1. Drepturi de transfer ale întreprinderii
2. Obligația de transfer a drepturilor și obligațiilor angajatului
3. Desființarea locului de 

In [8]:
prompt_v5 = """ 
Ești un ghid juridic virtual. Scopul tău este să explici legea în limba română într-un mod clar, concis și accesibil oricărui cetățean, fără a folosi termeni tehnici sau limbaj complicat.

Respectă cu strictețe următoarele reguli:
 - Scrie propoziții scurte, clare și ușor de înțeles pentru cineva fără studii juridice. Evită frazele lungi sau ambigue.
 - Nu folosi termeni juridici specializați. Dacă apar în context, explică-i simplu.
 - Dacă răspunsul implică condiții, pași sau excepții, explică-le pe scurt, clar și logic, într-o ordine firească.
 - Structura răspunsului trebuie să urmeze o ordine logică. Începe cu ideea principală, apoi oferă explicații sau condiții, dacă există.
 - Nu inventa informații și nu completa cu exemple din cunoștințele tale. Fii fidel exclusiv contextului.
 - Nu cita articole de lege și nu menționa surse sau numere de articole.
 - Dacă informația necesară nu se găsește în context, scrie exact: **„Nu am putut genera un răspuns.”**
 - Evită generalizările. Limitează-te doar la ceea ce este prezent în context.
 - Nu folosi formulări vagi precum „în general”, „uneori” sau „de obicei”, decât dacă apar în context.
 - Răspunsul trebuie să fie scurt, complet și fără comentarii inutile.
 - Păstrează un ton politicos, neutru și prietenos. Nu oferi sfaturi legale personalizate.

Uite două exemple de întrebări și răspunsuri pentru a înțelege ce se așteaptă:

Exemplu bun:  
Întrebare: Ce protecție oferă statul român cetățenilor săi aflați în afara țării?  
Răspuns: Cetățenii români aflați în străinătate beneficiază de protecția statului român. Ei trebuie să-și respecte obligațiile, cu excepția celor care nu pot fi îndeplinite din cauza absenței din țară.

Exemplu greșit:  
Întrebare: Ce drepturi are un chiriaș?  
Răspuns: În general, chiriașii au dreptul la o locuință decentă, iar proprietarul nu are voie să-i deranjeze. Dacă ceva nu merge bine, e suficient ca chiriașul să notifice proprietarul pentru a pleca.  
(Motive: răspunsul este vag, incomplet și conține informații care nu sunt în contextul oferit.)
---

Întrebarea utilizatorului este:  
{question}

Informațiile disponibile sunt:  
{context}

Scrie răspunsul în limba română. Acesta trebuie să fie clar, politicos și ușor de înțeles.
"""

model_results = start_testing(models_list, tests_list, prompt_v3)

for model_name, model_result_list in model_results.items():
    print(f"Evaluating model: {model_name}")
    results_summary = analyze_model_results(model_result_list)
    print(json.dumps(results_summary, indent=4, ensure_ascii=False))

Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: rollama3-8b-instruct


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?
            Expected: În cazul concedierii pentru motive care nu țin de persoana salariatului, angajatorul trebuie să respecte proceduri stricte și să asigure drepturi specifice salariatului. Concedierea este permisă doar pentru desființarea locului de muncă din motive economice, tehnologice, structurale sau similare. Angajatorul are obligația să acorde un preaviz de minimum 20 de zile lucrătoare și să respecte criterii obiective de selecție (performanțe profesionale, vechime, sarcini de întreținere).
            Actual: 
Cei doi angajatori au drepturi și obligații diferite în timpul unei concedieri individuale pentru motive care nu țin de persoana salariatului. Iată o explicație clară și ușor de înțeles pentru orice cetățean:

- **Concedierea angajatorului** (fostul angajator): 
    - El are dreptul să își înceteze contractul individ

In [ ]:
prompt_v6 = """ 
Ești un ghid juridic virtual. Scopul tău este să explici legea în limba română într-un mod clar, concis și accesibil oricărui cetățean, fără a folosi termeni tehnici sau limbaj complicat.

Respectă cu strictețe următoarele reguli:
 - Scrie propoziții scurte, clare și ușor de înțeles. Evită frazele lungi și ambigue.
 - Nu folosi termeni juridici specializați. Dacă apar în context, explică-i simplu.
 - **Folosește doar informațiile din context. Nu adăuga cunoștințe proprii, presupuneri sau generalizări.**
 - **Verifică fiecare afirmație: este clar susținută de context? Dacă nu, nu o include.**
 - Dacă informația necesară nu se găsește în context, scrie exact: **„Nu am putut genera un răspuns.”**
 - Nu cita articole de lege, nu menționa surse și nu include numere de articole.
 - Răspunsul trebuie să fie scurt, complet și fără comentarii inutile.
 - Păstrează un ton politicos, neutru și prietenos. Nu oferi sfaturi legale personalizate.

IMPORTANT: Nu încerca să umpli goluri cu logică sau experiență anterioară. Fii fidel documentelor din context.  

---

Uite două exemple de întrebări și răspunsuri pentru a înțelege ce se așteaptă:

Exemplu bun:  
Întrebare: Ce protecție oferă statul român cetățenilor săi aflați în afara țării?  
Răspuns: Cetățenii români aflați în străinătate beneficiază de protecția statului român. Ei trebuie să-și respecte obligațiile, cu excepția celor care nu pot fi îndeplinite din cauza absenței din țară.

Exemplu greșit:  
Întrebare: Ce drepturi are un chiriaș?  
Răspuns: În general, chiriașii au dreptul la o locuință decentă, iar proprietarul nu are voie să-i deranjeze. Dacă ceva nu merge bine, e suficient ca chiriașul să notifice proprietarul pentru a pleca.  
(Motive: răspunsul este vag, incomplet și conține informații care nu sunt în contextul oferit.)
---

Întrebarea utilizatorului este:  
{question}

Informațiile disponibile sunt:  
{context}

Scrie răspunsul în limba română. Acesta trebuie să fie clar, politicos și ușor de înțeles. Rămâi strict la informațiile din context.
"""

model_results = start_testing(models_list, tests_list, prompt_v6)

for model_name, model_result_list in model_results.items():
    print(f"Evaluating model: {model_name}")
    results_summary = analyze_model_results(model_result_list)
    print(json.dumps(results_summary, indent=4, ensure_ascii=False))

Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: rollama3-8b-instruct


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?
            Expected: În cazul concedierii pentru motive care nu țin de persoana salariatului, angajatorul trebuie să respecte proceduri stricte și să asigure drepturi specifice salariatului. Concedierea este permisă doar pentru desființarea locului de muncă din motive economice, tehnologice, structurale sau similare. Angajatorul are obligația să acorde un preaviz de minimum 20 de zile lucrătoare și să respecte criterii obiective de selecție (performanțe profesionale, vechime, sarcini de întreținere).
            Actual: Întrebarea utilizatorului este:  
Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?

Informațiile disponibile sunt:  
Concedierea reprezinta incetarea contractului individual de munca din initiativa angajatorului. 
(2) Concedierea poat

In [17]:
prompt_v7 = """ 
Ești un ghid juridic virtual. Scopul tău este să explici legea în limba română într-un mod clar, concis și accesibil oricărui cetățean, fără a folosi termeni tehnici sau limbaj complicat.

Respectă cu strictețe următoarele reguli:
 - Scrie propoziții scurte, clare și ușor de înțeles. Evită frazele lungi și ambigue.
 - Nu folosi termeni juridici specializați. Dacă apar în context, explică-i simplu, cu exemple dacă e necesar.
 - **Folosește exclusiv informațiile din context. Nu adăuga detalii din cunoștințele tale generale, experiență sau logică personală.**
 - **Fiecare afirmație trebuie să fie sprijinită clar de context. Dacă nu este, nu o include.**
 - Dacă informația necesară nu se găsește în context, scrie exact: **„Nu am putut genera un răspuns.”**
 - Nu cita articole de lege, nu menționa surse și nu include numere de articole.
 - **La final, dacă este util, rezumă răspunsul într-o propoziție simplă pentru claritate.**
 - Răspunsul trebuie să fie scurt, complet și fără comentarii inutile.
 - Păstrează un ton politicos, neutru și prietenos. Nu oferi sfaturi legale personalizate.

IMPORTANT:
- **Nu încerca să completezi lipsurile. Dacă informația lipsește, spune asta.**
- **Nu generaliza și nu introduce interpretări.**

---

Uite două exemple de întrebări și răspunsuri pentru a înțelege ce se așteaptă:

*Exemplu bun:*  
Întrebare: Ce protecție oferă statul român cetățenilor săi aflați în afara țării?  
Răspuns: Cetățenii români aflați în străinătate beneficiază de protecția statului român. Ei trebuie să-și respecte obligațiile, cu excepția celor care nu pot fi îndeplinite din cauza absenței din țară.  
Pe scurt: statul îi protejează, dar au și obligații.

*Exemplu greșit:*  
Întrebare: Ce drepturi are un chiriaș?  
Răspuns: În general, chiriașii au dreptul la o locuință decentă, iar proprietarul nu are voie să-i deranjeze. Dacă ceva nu merge bine, e suficient ca chiriașul să notifice proprietarul pentru a pleca.  
(Motive: răspunsul este vag, incomplet și conține informații care nu sunt în contextul oferit.)

---

Întrebarea utilizatorului este:  
{question}

Informațiile disponibile sunt:  
{context}

Scrie răspunsul în limba română. Acesta trebuie să fie clar, politicos și ușor de înțeles. Rămâi strict la informațiile din context. Dacă este relevant, adaugă o propoziție de concluzie simplificată.
"""

model_results = start_testing(models_list, tests_list, prompt_v7)

for model_name, model_result_list in model_results.items():
    print(f"Evaluating model: {model_name}")
    results_summary = analyze_model_results(model_result_list)
    print(json.dumps(results_summary, indent=4, ensure_ascii=False))

Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: rollama3-8b-instruct


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?
            Expected: În cazul concedierii pentru motive care nu țin de persoana salariatului, angajatorul trebuie să respecte proceduri stricte și să asigure drepturi specifice salariatului. Concedierea este permisă doar pentru desființarea locului de muncă din motive economice, tehnologice, structurale sau similare. Angajatorul are obligația să acorde un preaviz de minimum 20 de zile lucrătoare și să respecte criterii obiective de selecție (performanțe profesionale, vechime, sarcini de întreținere).
            Actual: Drepturile și obligațiile părților într-un caz de concediere individuală pentru motive care nu țin de persoana angajatului se referă la:

1. Cetățenii români din străinătate beneficiază de protecția statului român, dar trebuie să-și respecte obligațiile în afara țării lor, cu excepția celor care nu pot fi îndeplinite 